In [1]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
def convert_dict_to_string(value):
    if isinstance(value, dict):
        return json.dumps(value)
    else:
        return value
def Read_data(url):
  response = requests.get(url)
  if response.status_code == 200:
     data = response.json()
     return data
  else:
    return print("Failed to fetch data from the API. Status code:", response.status_code)

In [3]:
#leemos la informacion
data=Read_data('https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow')

In [4]:
#Exploracion de los datos
data

{'items': [{'tags': ['java', 'perl'],
   'owner': {'reputation': 11547,
    'user_id': 307297,
    'user_type': 'registered',
    'accept_rate': 62,
    'profile_image': 'https://i.stack.imgur.com/e28FD.png?s=256&g=1',
    'display_name': 'chris01',
    'link': 'https://stackoverflow.com/users/307297/chris01'},
   'is_answered': False,
   'view_count': 33,
   'answer_count': 0,
   'score': 0,
   'last_activity_date': 1710544120,
   'creation_date': 1710506166,
   'last_edit_date': 1710544120,
   'question_id': 78166968,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/78166968/inlinejava-inside-of-perl-threads-unknown-java-object-reference',
   'title': 'Inline::Java inside of Perl Threads: unknown java object reference)'},
  {'tags': ['xml', 'perl', 'parsing', 'templates'],
   'owner': {'reputation': 1,
    'user_id': 1883796,
    'user_type': 'registered',
    'profile_image': 'https://www.gravatar.com/avatar/6fe26b3696a53618f2d55d82a3b57b33?s=256

In [5]:
#se convierte a DataFrame
lt=data['items']
df_t=pd.DataFrame(lt)

In [6]:
#visualizacion del DataFrame
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tags                30 non-null     object 
 1   owner               30 non-null     object 
 2   is_answered         30 non-null     bool   
 3   view_count          30 non-null     int64  
 4   answer_count        30 non-null     int64  
 5   score               30 non-null     int64  
 6   last_activity_date  30 non-null     int64  
 7   creation_date       30 non-null     int64  
 8   last_edit_date      22 non-null     float64
 9   question_id         30 non-null     int64  
 10  content_license     25 non-null     object 
 11  link                30 non-null     object 
 12  title               30 non-null     object 
 13  closed_date         5 non-null      float64
 14  closed_reason       5 non-null      object 
 15  accepted_answer_id  13 non-null     float64
 16  protected_

In [ ]:
#se corrige los formatos de tiempo
df_t['last_activity_date']=pd.to_datetime(df_t['last_activity_date'], unit='s')
df_t['creation_date']=pd.to_datetime(df_t['creation_date'], unit='s')


In [8]:
#la columna Owner tiene datos anidados, es necesario incorporarlos a la tabla para el analisis
df_t['owner'].head()


0    {'reputation': 11547, 'user_id': 307297, 'user...
1    {'reputation': 1, 'user_id': 1883796, 'user_ty...
2    {'reputation': 3921, 'user_id': 6607497, 'user...
3    {'reputation': 190, 'user_id': 5714031, 'user_...
4    {'reputation': 49, 'user_id': 1358240, 'user_t...
Name: owner, dtype: object

In [9]:
#Se recupera la informacion de la columna Owner y se concatena junto con el DataFrame original en un nuevo DataFrame
df_t['owner'] = df_t['owner'].apply(convert_dict_to_string)
df_t['owner'] = df_t['owner'].apply(json.loads)
json_df = pd.json_normalize(df_t['owner'])
df = df_t.drop(columns=['owner'])
df = pd.concat([df, json_df], axis=1)

Exploracion del nuevo DataFrame

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tags                30 non-null     object        
 1   is_answered         30 non-null     bool          
 2   view_count          30 non-null     int64         
 3   answer_count        30 non-null     int64         
 4   score               30 non-null     int64         
 5   last_activity_date  30 non-null     datetime64[ns]
 6   creation_date       30 non-null     datetime64[ns]
 7   last_edit_date      22 non-null     datetime64[ns]
 8   question_id         30 non-null     int64         
 9   content_license     25 non-null     object        
 10  link                30 non-null     object        
 11  title               30 non-null     object        
 12  closed_date         5 non-null      datetime64[ns]
 13  closed_reason       5 non-null      object        
 

In [11]:
df.head(3)

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,closed_reason,accepted_answer_id,protected_date,reputation,user_id,user_type,accept_rate,profile_image,display_name,link
0,"[java, perl]",False,33,0,0,2024-03-15 23:08:40,2024-03-15 12:36:06,2024-03-15 23:08:40,78166968,CC BY-SA 4.0,...,NaN,NaN,NaT,11547,307297,registered,62.0,https://i.stack.imgur.com/e28FD.png?s=256&g=1,chris01,https://stackoverflow.com/users/307297/chris01
1,"[xml, perl, parsing, templates]",False,48,1,-4,2024-03-15 13:33:02,2024-03-14 04:04:38,2024-03-15 13:33:02,78158052,NaN,...,Needs details or clarity,NaN,NaT,1,1883796,registered,NaN,https://www.gravatar.com/avatar/6fe26b3696a536...,user1883796,https://stackoverflow.com/users/1883796/user18...
2,"[perl, floating-point, ieee-754]",True,87,1,1,2024-03-15 13:19:02,2024-03-14 14:28:38,NaT,78161270,CC BY-SA 4.0,...,NaN,NaN,NaT,3921,6607497,registered,17.0,https://www.gravatar.com/avatar/4ec084e75bd895...,U. Windl,https://stackoverflow.com/users/6607497/u-windl


2. Obtener el número de respuestas contestadas y no contestadas


In [24]:
respuestas = df['is_answered'].value_counts()
print("Respuestas contestadas:", respuestas[True], "Respuestas No contestadas:", respuestas[False])

Respuestas contestadas: 20 Respuestas No contestadas: 10


3. Obtener la respuesta con menor número de vistas

In [14]:
min_val = df['view_count'].min()
print("Minimum value in column:", min_val)

Minimum value in column: 4


4. Obtener la respuesta más vieja y más actual


In [23]:
min_val = df['creation_date'].min()
max_val=df['creation_date'].max()
print("Respuesta mas vieja: ", min_val, "Respuesta mas reciente: ",max_val)

Respuesta mas vieja:  2008-08-11 10:04:32 Respuesta mas reciente:  2024-03-15 12:36:06


5. Obtener la respuesta del owner que tenga una mayor reputación


In [22]:
print(df[df['reputation'] ==df['reputation'].max()],"Owner Reputation: ",df['reputation'].max())

                    tags  is_answered  view_count  answer_count  score  \
5  [perl, nan, infinity]         True       14337             7     26   

   last_activity_date       creation_date last_edit_date  question_id  \
5 2024-03-14 14:00:34 2009-07-26 23:05:08            NaT      1185822   

  content_license  ... closed_reason accepted_answer_id protected_date  \
5    CC BY-SA 2.5  ...           NaN          1185828.0            NaT   

  reputation  user_id   user_type  accept_rate  \
5      97070    17389  registered         38.0   

                                       profile_image display_name  \
5  https://www.gravatar.com/avatar/cd06d1314817ce...         ysth   

                                         link  
5  https://stackoverflow.com/users/17389/ysth  

[1 rows x 23 columns] Owner Reputation:  97070
